In [26]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA
import tensorflow as tf
from sklearn.model_selection import train_test_split
%matplotlib inline

In [32]:
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [3]:
train_df.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [33]:
y_train = train_df['label']
train_df.drop('label', inplace=True, axis=1)

In [5]:
train_df.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# 1. Метод k-ближайших соседей

In [13]:
knn_params = {'n_neighbors': list(range(2, 10)) + list(range(10, 30, 5)) + list(range(30, 100, 10))}
knn_grid = GridSearchCV(KNeighborsClassifier(n_jobs=-1), param_grid=knn_params, cv=2)

In [14]:
%%time
knn_grid.fit(train_df, y_train)

CPU times: user 1min 41s, sys: 5.51 s, total: 1min 46s
Wall time: 1h 22min 43s


GridSearchCV(cv=2, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25,
                                         30, 40, 50, 60, 70, 80, 90]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [16]:
knn_grid.best_params_, knn_grid.best_score_

({'n_neighbors': 3}, 0.9604761904761905)

In [21]:
y_pred = knn_grid.predict(test_df)

In [22]:
pd.Series(y_pred, index=range(1, y_pred.shape[0]+1), name='Label').to_csv('submission.csv', header=True, index_label='ImageId')

In [29]:
knn_grid2 = GridSearchCV(KNeighborsClassifier(n_jobs=-1), param_grid={'n_neighbors': [3]} ,cv=4)
knn_grid2.fit(train_df, y_train)

GridSearchCV(cv=4, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=-1,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None, param_grid={'n_neighbors': [3]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [30]:
knn_grid2.best_params_, knn_grid2.best_score_

({'n_neighbors': 3}, 0.9660952380952381)

# 2. Случайный лес

In [6]:
rand_params = {'n_estimators': range(100, 600, 100), 'max_depth': list(range(2,5)) + list(range(5, 50, 10)),
              'max_features': [392, 450, 500, 650, 700, 784]}
rand_forest = GridSearchCV(RandomForestClassifier(n_jobs=-1), param_grid=rand_params, cv=4)

In [27]:
%%time
rand_forest = RandomForestClassifier(n_jobs=-1, n_estimators=1000, max_depth=10)
rand_forest.fit(train_df, y_train)

CPU times: user 3min 48s, sys: 292 ms, total: 3min 48s
Wall time: 29.9 s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [28]:
rand_forest.score(train_df, y_train)

0.971

In [4]:
X_train, X_test, y_tr, y_ts = train_test_split(train_df, y_train, test_size=0.2)

In [29]:
rand_forest.fit(X_train, y_tr)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=10, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=1000,
                       n_jobs=-1, oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [30]:
rand_forest.score(X_train, y_tr)

0.9747916666666666

In [31]:
y_pr = rand_forest.predict(X_test)
accuracy_score(y_pr, y_ts)

0.9463095238095238

In [32]:
knn = KNeighborsClassifier(n_neighbors=3, n_jobs=-1)
knn.fit(X_train, y_tr)
knn.score(X_train, y_tr)

0.9822321428571429

In [ ]:
accuracy_score(knn.predict(X_test), y_ts)

# Нейронные сети

In [67]:
model = tf.keras.Sequential([tf.keras.layers.Flatten(input_shape=(28, 28)),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dense(256, activation='relu'),
                             tf.keras.layers.Dropout(0.1),
                             tf.keras.layers.Dense(10, activation='softmax')])
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [25]:
pd.Series(y_pred, index=range(1, y_pred.shape[0]+1), name='Label').to_csv('submission.csv', header=True, index_label='ImageId')

In [34]:
X_train, X_test, y_train, y_test = train_test_split(train_df, y_train, test_size=0.3)

In [69]:
model.fit(X_train, y_train, epochs=10)
model.evaluate(X_test,  y_test, verbose=2)

Epoch 1/10
919/919 [==============================] - 6s 6ms/step - loss: 0.5993 - accuracy: 0.8268
Epoch 2/10
919/919 [==============================] - 6s 6ms/step - loss: 0.6546 - accuracy: 0.8149
Epoch 3/10
919/919 [==============================] - 6s 6ms/step - loss: 0.7864 - accuracy: 0.7567
Epoch 4/10
919/919 [==============================] - 6s 6ms/step - loss: 0.8046 - accuracy: 0.7308
Epoch 5/10
919/919 [==============================] - 6s 6ms/step - loss: 0.6151 - accuracy: 0.8002
Epoch 6/10
919/919 [==============================] - 6s 6ms/step - loss: 0.5232 - accuracy: 0.8617
Epoch 7/10
919/919 [==============================] - 6s 6ms/step - loss: 0.6062 - accuracy: 0.8391
Epoch 8/10
919/919 [==============================] - 6s 6ms/step - loss: 0.6777 - accuracy: 0.8031
Epoch 9/10
741/919 [=======================>......] - ETA: 1s - loss: 0.7080 - accuracy: 0.7894

KeyboardInterrupt: 

In [64]:
y_pred = model.predict_classes(test_df)
pd.Series(y_pred, index=range(1, y_pred.shape[0]+1), name='Label').to_csv('submission.csv', header=True, index_label='ImageId')